In [ ]:
## import (and install if necessary) pandas
%pip install pandas
import pandas as pd

In [2]:
df = pd.read_csv("full_data.csv"
                 ,parse_dates=['TripDate','StartDateTime','EndDateTime','CustomerBirthDate']
                 )
## process cost monetary data
df['Cost'] = df['Cost'].str.replace(',', '')
df['Cost'] = df['Cost'].str.replace('$', '')
df['Cost'] = df['Cost'].str.replace('(', '')
df['Cost'] = df['Cost'].str.replace(')', '')
df['Cost'] = df['Cost'].astype(float)
print(df.info())
#print(df.head(5))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1160755 entries, 0 to 1160754
Data columns (total 14 columns):
 #   Column             Non-Null Count    Dtype         
---  ------             --------------    -----         
 0   CustomerID         1160755 non-null  int64         
 1   CustomerSex        1160755 non-null  object        
 2   CustomerBirthDate  1160417 non-null  datetime64[ns]
 3   TripDate           1160755 non-null  datetime64[ns]
 4   VehicleID          1160755 non-null  int64         
 5   RunActualID        1160755 non-null  int64         
 6   StartDateTime      1160755 non-null  datetime64[ns]
 7   EndDateTime        1160755 non-null  datetime64[ns]
 8   PickupX            1110755 non-null  float64       
 9   PickupY            1110755 non-null  float64       
 10  DropoffX           1109028 non-null  float64       
 11  DropoffY           1109028 non-null  float64       
 12  Cost               1160538 non-null  float64       
 13  CollectedFareType  1147941 

In [3]:
## summary counts
runActualID_counts = df.groupby(['RunActualID'])['RunActualID'].count()
max_runActualIDs = runActualID_counts.max().item()
print(f"Maximum Number of rows per RunActualID: {max_runActualIDs}")
runActualID_vehicleID = df.groupby('RunActualID')['VehicleID'].nunique()
runActualIDs_more_than_one_vehicleID = runActualID_vehicleID[runActualID_vehicleID != 1]
print(f"Any RunActualIDs that have more than one VehicleID: {runActualIDs_more_than_one_vehicleID.nunique()}")
print(f"Unique number of RunActualIDs: {df['RunActualID'].nunique()}")
print(f"Unique number of CustomerIDs: {df['CustomerID'].nunique()}")
print(f"Unique number of VehicleIDs: {df['VehicleID'].nunique()}")

Maximum Number of rows per RunActualID: 18
Any RunActualIDs that have more than one VehicleID: 0
Unique number of RunActualIDs: 213272
Unique number of CustomerIDs: 10215
Unique number of VehicleIDs: 538


In [4]:
dates_per_runactualIDs = df.groupby(['RunActualID']).agg({'EndDateTime': 'max', 'StartDateTime':'min'})
dates_per_runactualIDs['EndDate'] = pd.to_datetime(dates_per_runactualIDs['EndDateTime'].dt.date)
dates_per_runactualIDs['StartDate'] = pd.to_datetime(dates_per_runactualIDs['StartDateTime'].dt.date)
dates_per_runactualIDs['diff'] = (dates_per_runactualIDs['EndDateTime'] - dates_per_runactualIDs['StartDateTime']).dt.total_seconds()
seconds_in_day = 86400
runactualIDs_over_day = dates_per_runactualIDs[dates_per_runactualIDs['diff'] > seconds_in_day]
#print(runactualIDs_over_day['RunActualId'].nunique())

print(f"""Number of RunActualIDs lasting over 24 hours: {runactualIDs_over_day.index.nunique()}
(Calculated by subtracting EndDateTime from StartDateTime)
Saving outlier RunActualIDs to `runactualIDs_over_day.csv""")
runactualIDs_over_day.to_csv("runactualIDs_over_day.csv", columns = ['StartDateTime','EndDateTime'])

Number of RunActualIDs lasting over 24 hours: 4
(Calculated by subtracting EndDateTime from StartDateTime)
Saving outlier RunActualIDs to `runactualIDs_over_day.csv


In [5]:
# adding in seconds per trip
df['TotalTripTimeSeconds'] = (df['EndDateTime'] - df['StartDateTime']).dt.total_seconds()


In [6]:
# sort by RunActualID and StartDateTime, add ride rank
df = df.sort_values(by=['RunActualID','StartDateTime'])
df = df.reset_index()
df['TripRank'] = df.sort_values(['RunActualID','StartDateTime'], ascending=True).groupby(['RunActualID']).cumcount() + 1


df_route_fields = df[['RunActualID','CustomerID','StartDateTime','PickupX','PickupY','EndDateTime','DropoffX','DropoffY','Cost','CollectedFareType','TotalTripTimeSeconds']]

In [ ]:
# generate field definitions
"""field_tuple = ()
for x in range(max_runActualIDs):
    y = x + 1
    field_tuple = field_tuple + (f"CustomerID_{y}=pd.NamedAgg(column='CustomerID',aggfunc = lambda t: t.head({y}).tail(1))",
                                 f"StartDateTime_{y}=pd.NamedAgg(column='StartDateTime',aggfunc = lambda t: t.head({y}).tail(1))",
                                 f"PickupX_{y}=pd.NamedAgg(column='PickupX',aggfunc = lambda t: t.head({y}).tail(1))",
                                 f"PickupY_{y}=pd.NamedAgg(column='PickupY',aggfunc = lambda t: t.head({y}).tail(1))",
                                 f"EndDateTime_{y}=pd.NamedAgg(column='EndDateTime',aggfunc = lambda t: t.head({y}).tail(1))",
                                 f"DropoffX_{y}=pd.NamedAgg(column='DropoffX',aggfunc = lambda t: t.head({y}).tail(1))",
                                 f"DropoffY_{y}=pd.NamedAgg(column='DropoffY',aggfunc = lambda t: t.head({y}).tail(1))",
                                 f"Cost_{y}=pd.NamedAgg(column='Cost',aggfunc = lambda t: t.head({y}).tail(1))",
                                 f"CollectedFareType_{y}=pd.NamedAgg(column='CollectedFareType',aggfunc = lambda t: t.head({y}).tail(1))",
                                 f"TotalTripTimeSeconds_{y}=pd.NamedAgg(column='TotalTripTimeSeconds',aggfunc = lambda t: t.head({y}).tail(1))"
                                 )
print(field_tuple)
"""

In [8]:


# there is 100% a better way to do this, just not today:
df_routes = df_route_fields.groupby('RunActualID').agg(
    TotalCustomers=pd.NamedAgg(column='CustomerID', aggfunc='count'),
    CustomerID_1=pd.NamedAgg(column='CustomerID',aggfunc = lambda t: t.head(1).tail(1)),
    StartDateTime_1=pd.NamedAgg(column='StartDateTime',aggfunc = lambda t: t.head(1).tail(1)),
    PickupX_1=pd.NamedAgg(column='PickupX',aggfunc = lambda t: t.head(1).tail(1)),
    PickupY_1=pd.NamedAgg(column='PickupY',aggfunc = lambda t: t.head(1).tail(1)),
    EndDateTime_1=pd.NamedAgg(column='EndDateTime',aggfunc = lambda t: t.head(1).tail(1)),
    DropoffX_1=pd.NamedAgg(column='DropoffX',aggfunc = lambda t: t.head(1).tail(1)),
    DropoffY_1=pd.NamedAgg(column='DropoffY',aggfunc = lambda t: t.head(1).tail(1)),
    Cost_1=pd.NamedAgg(column='Cost',aggfunc = lambda t: t.head(1).tail(1)),
    CollectedFareType_1=pd.NamedAgg(column='CollectedFareType',aggfunc = lambda t: t.head(1).tail(1)),
    TotalTripTimeSeconds_1=pd.NamedAgg(column='TotalTripTimeSeconds',aggfunc = lambda t: t.head(1).tail(1)),
    CustomerID_2=pd.NamedAgg(column='CustomerID',aggfunc = lambda t: t.head(2).tail(1)),
    StartDateTime_2=pd.NamedAgg(column='StartDateTime',aggfunc = lambda t: t.head(2).tail(1)),
    PickupX_2=pd.NamedAgg(column='PickupX',aggfunc = lambda t: t.head(2).tail(1)),
    PickupY_2=pd.NamedAgg(column='PickupY',aggfunc = lambda t: t.head(2).tail(1)),
    EndDateTime_2=pd.NamedAgg(column='EndDateTime',aggfunc = lambda t: t.head(2).tail(1)),
    DropoffX_2=pd.NamedAgg(column='DropoffX',aggfunc = lambda t: t.head(2).tail(1)),
    DropoffY_2=pd.NamedAgg(column='DropoffY',aggfunc = lambda t: t.head(2).tail(1)),
    Cost_2=pd.NamedAgg(column='Cost',aggfunc = lambda t: t.head(2).tail(1)),
    CollectedFareType_2=pd.NamedAgg(column='CollectedFareType',aggfunc = lambda t: t.head(2).tail(1)),
    TotalTripTimeSeconds_2=pd.NamedAgg(column='TotalTripTimeSeconds',aggfunc = lambda t: t.head(2).tail(1)),
    CustomerID_3=pd.NamedAgg(column='CustomerID',aggfunc = lambda t: t.head(3).tail(1)),
    StartDateTime_3=pd.NamedAgg(column='StartDateTime',aggfunc = lambda t: t.head(3).tail(1)),
    PickupX_3=pd.NamedAgg(column='PickupX',aggfunc = lambda t: t.head(3).tail(1)),
    PickupY_3=pd.NamedAgg(column='PickupY',aggfunc = lambda t: t.head(3).tail(1)),
    EndDateTime_3=pd.NamedAgg(column='EndDateTime',aggfunc = lambda t: t.head(3).tail(1)),
    DropoffX_3=pd.NamedAgg(column='DropoffX',aggfunc = lambda t: t.head(3).tail(1)),
    DropoffY_3=pd.NamedAgg(column='DropoffY',aggfunc = lambda t: t.head(3).tail(1)),
    Cost_3=pd.NamedAgg(column='Cost',aggfunc = lambda t: t.head(3).tail(1)),
    CollectedFareType_3=pd.NamedAgg(column='CollectedFareType',aggfunc = lambda t: t.head(3).tail(1)),
    TotalTripTimeSeconds_3=pd.NamedAgg(column='TotalTripTimeSeconds',aggfunc = lambda t: t.head(3).tail(1)),
    CustomerID_4=pd.NamedAgg(column='CustomerID',aggfunc = lambda t: t.head(4).tail(1)),
    StartDateTime_4=pd.NamedAgg(column='StartDateTime',aggfunc = lambda t: t.head(4).tail(1)),
    PickupX_4=pd.NamedAgg(column='PickupX',aggfunc = lambda t: t.head(4).tail(1)),
    PickupY_4=pd.NamedAgg(column='PickupY',aggfunc = lambda t: t.head(4).tail(1)),
    EndDateTime_4=pd.NamedAgg(column='EndDateTime',aggfunc = lambda t: t.head(4).tail(1)),
    DropoffX_4=pd.NamedAgg(column='DropoffX',aggfunc = lambda t: t.head(4).tail(1)),
    DropoffY_4=pd.NamedAgg(column='DropoffY',aggfunc = lambda t: t.head(4).tail(1)),
    Cost_4=pd.NamedAgg(column='Cost',aggfunc = lambda t: t.head(4).tail(1)),
    CollectedFareType_4=pd.NamedAgg(column='CollectedFareType',aggfunc = lambda t: t.head(4).tail(1)),
    TotalTripTimeSeconds_4=pd.NamedAgg(column='TotalTripTimeSeconds',aggfunc = lambda t: t.head(4).tail(1)),
    CustomerID_5=pd.NamedAgg(column='CustomerID',aggfunc = lambda t: t.head(5).tail(1)),
    StartDateTime_5=pd.NamedAgg(column='StartDateTime',aggfunc = lambda t: t.head(5).tail(1)),
    PickupX_5=pd.NamedAgg(column='PickupX',aggfunc = lambda t: t.head(5).tail(1)),
    PickupY_5=pd.NamedAgg(column='PickupY',aggfunc = lambda t: t.head(5).tail(1)),
    EndDateTime_5=pd.NamedAgg(column='EndDateTime',aggfunc = lambda t: t.head(5).tail(1)),
    DropoffX_5=pd.NamedAgg(column='DropoffX',aggfunc = lambda t: t.head(5).tail(1)),
    DropoffY_5=pd.NamedAgg(column='DropoffY',aggfunc = lambda t: t.head(5).tail(1)),
    Cost_5=pd.NamedAgg(column='Cost',aggfunc = lambda t: t.head(5).tail(1)),
    CollectedFareType_5=pd.NamedAgg(column='CollectedFareType',aggfunc = lambda t: t.head(5).tail(1)),
    TotalTripTimeSeconds_5=pd.NamedAgg(column='TotalTripTimeSeconds',aggfunc = lambda t: t.head(5).tail(1)),
    CustomerID_6=pd.NamedAgg(column='CustomerID',aggfunc = lambda t: t.head(6).tail(1)),
    StartDateTime_6=pd.NamedAgg(column='StartDateTime',aggfunc = lambda t: t.head(6).tail(1)),
    PickupX_6=pd.NamedAgg(column='PickupX',aggfunc = lambda t: t.head(6).tail(1)),
    PickupY_6=pd.NamedAgg(column='PickupY',aggfunc = lambda t: t.head(6).tail(1)),
    EndDateTime_6=pd.NamedAgg(column='EndDateTime',aggfunc = lambda t: t.head(6).tail(1)),
    DropoffX_6=pd.NamedAgg(column='DropoffX',aggfunc = lambda t: t.head(6).tail(1)),
    DropoffY_6=pd.NamedAgg(column='DropoffY',aggfunc = lambda t: t.head(6).tail(1)),
    Cost_6=pd.NamedAgg(column='Cost',aggfunc = lambda t: t.head(6).tail(1)),
    CollectedFareType_6=pd.NamedAgg(column='CollectedFareType',aggfunc = lambda t: t.head(6).tail(1)),
    TotalTripTimeSeconds_6=pd.NamedAgg(column='TotalTripTimeSeconds',aggfunc = lambda t: t.head(6).tail(1)),
    CustomerID_7=pd.NamedAgg(column='CustomerID',aggfunc = lambda t: t.head(7).tail(1)),
    StartDateTime_7=pd.NamedAgg(column='StartDateTime',aggfunc = lambda t: t.head(7).tail(1)),
    PickupX_7=pd.NamedAgg(column='PickupX',aggfunc = lambda t: t.head(7).tail(1)),
    PickupY_7=pd.NamedAgg(column='PickupY',aggfunc = lambda t: t.head(7).tail(1)),
    EndDateTime_7=pd.NamedAgg(column='EndDateTime',aggfunc = lambda t: t.head(7).tail(1)),
    DropoffX_7=pd.NamedAgg(column='DropoffX',aggfunc = lambda t: t.head(7).tail(1)),
    DropoffY_7=pd.NamedAgg(column='DropoffY',aggfunc = lambda t: t.head(7).tail(1)),
    Cost_7=pd.NamedAgg(column='Cost',aggfunc = lambda t: t.head(7).tail(1)),
    CollectedFareType_7=pd.NamedAgg(column='CollectedFareType',aggfunc = lambda t: t.head(7).tail(1)),
    TotalTripTimeSeconds_7=pd.NamedAgg(column='TotalTripTimeSeconds',aggfunc = lambda t: t.head(7).tail(1)),
    CustomerID_8=pd.NamedAgg(column='CustomerID',aggfunc = lambda t: t.head(8).tail(1)),
    StartDateTime_8=pd.NamedAgg(column='StartDateTime',aggfunc = lambda t: t.head(8).tail(1)),
    PickupX_8=pd.NamedAgg(column='PickupX',aggfunc = lambda t: t.head(8).tail(1)),
    PickupY_8=pd.NamedAgg(column='PickupY',aggfunc = lambda t: t.head(8).tail(1)),
    EndDateTime_8=pd.NamedAgg(column='EndDateTime',aggfunc = lambda t: t.head(8).tail(1)),
    DropoffX_8=pd.NamedAgg(column='DropoffX',aggfunc = lambda t: t.head(8).tail(1)),
    DropoffY_8=pd.NamedAgg(column='DropoffY',aggfunc = lambda t: t.head(8).tail(1)),
    Cost_8=pd.NamedAgg(column='Cost',aggfunc = lambda t: t.head(8).tail(1)),
    CollectedFareType_8=pd.NamedAgg(column='CollectedFareType',aggfunc = lambda t: t.head(8).tail(1)),
    TotalTripTimeSeconds_8=pd.NamedAgg(column='TotalTripTimeSeconds',aggfunc = lambda t: t.head(8).tail(1)),
    CustomerID_9=pd.NamedAgg(column='CustomerID',aggfunc = lambda t: t.head(9).tail(1)),
    StartDateTime_9=pd.NamedAgg(column='StartDateTime',aggfunc = lambda t: t.head(9).tail(1)),
    PickupX_9=pd.NamedAgg(column='PickupX',aggfunc = lambda t: t.head(9).tail(1)),
    PickupY_9=pd.NamedAgg(column='PickupY',aggfunc = lambda t: t.head(9).tail(1)),
    EndDateTime_9=pd.NamedAgg(column='EndDateTime',aggfunc = lambda t: t.head(9).tail(1)),
    DropoffX_9=pd.NamedAgg(column='DropoffX',aggfunc = lambda t: t.head(9).tail(1)),
    DropoffY_9=pd.NamedAgg(column='DropoffY',aggfunc = lambda t: t.head(9).tail(1)),
    Cost_9=pd.NamedAgg(column='Cost',aggfunc = lambda t: t.head(9).tail(1)),
    CollectedFareType_9=pd.NamedAgg(column='CollectedFareType',aggfunc = lambda t: t.head(9).tail(1)),
    TotalTripTimeSeconds_9=pd.NamedAgg(column='TotalTripTimeSeconds',aggfunc = lambda t: t.head(9).tail(1)),
    CustomerID_10=pd.NamedAgg(column='CustomerID',aggfunc = lambda t: t.head(10).tail(1)),
    StartDateTime_10=pd.NamedAgg(column='StartDateTime',aggfunc = lambda t: t.head(10).tail(1)),
    PickupX_10=pd.NamedAgg(column='PickupX',aggfunc = lambda t: t.head(10).tail(1)),
    PickupY_10=pd.NamedAgg(column='PickupY',aggfunc = lambda t: t.head(10).tail(1)),
    EndDateTime_10=pd.NamedAgg(column='EndDateTime',aggfunc = lambda t: t.head(10).tail(1)),
    DropoffX_10=pd.NamedAgg(column='DropoffX',aggfunc = lambda t: t.head(10).tail(1)),
    DropoffY_10=pd.NamedAgg(column='DropoffY',aggfunc = lambda t: t.head(10).tail(1)),
    Cost_10=pd.NamedAgg(column='Cost',aggfunc = lambda t: t.head(10).tail(1)),
    CollectedFareType_10=pd.NamedAgg(column='CollectedFareType',aggfunc = lambda t: t.head(10).tail(1)),
    TotalTripTimeSeconds_10=pd.NamedAgg(column='TotalTripTimeSeconds',aggfunc = lambda t: t.head(10).tail(1)),
    CustomerID_11=pd.NamedAgg(column='CustomerID',aggfunc = lambda t: t.head(11).tail(1)),
    StartDateTime_11=pd.NamedAgg(column='StartDateTime',aggfunc = lambda t: t.head(11).tail(1)),
    PickupX_11=pd.NamedAgg(column='PickupX',aggfunc = lambda t: t.head(11).tail(1)),
    PickupY_11=pd.NamedAgg(column='PickupY',aggfunc = lambda t: t.head(11).tail(1)),
    EndDateTime_11=pd.NamedAgg(column='EndDateTime',aggfunc = lambda t: t.head(11).tail(1)),
    DropoffX_11=pd.NamedAgg(column='DropoffX',aggfunc = lambda t: t.head(11).tail(1)),
    DropoffY_11=pd.NamedAgg(column='DropoffY',aggfunc = lambda t: t.head(11).tail(1)),
    Cost_11=pd.NamedAgg(column='Cost',aggfunc = lambda t: t.head(11).tail(1)),
    CollectedFareType_11=pd.NamedAgg(column='CollectedFareType',aggfunc = lambda t: t.head(11).tail(1)),
    TotalTripTimeSeconds_11=pd.NamedAgg(column='TotalTripTimeSeconds',aggfunc = lambda t: t.head(11).tail(1)),
    CustomerID_12=pd.NamedAgg(column='CustomerID',aggfunc = lambda t: t.head(12).tail(1)),
    StartDateTime_12=pd.NamedAgg(column='StartDateTime',aggfunc = lambda t: t.head(12).tail(1)),
    PickupX_12=pd.NamedAgg(column='PickupX',aggfunc = lambda t: t.head(12).tail(1)),
    PickupY_12=pd.NamedAgg(column='PickupY',aggfunc = lambda t: t.head(12).tail(1)),
    EndDateTime_12=pd.NamedAgg(column='EndDateTime',aggfunc = lambda t: t.head(12).tail(1)),
    DropoffX_12=pd.NamedAgg(column='DropoffX',aggfunc = lambda t: t.head(12).tail(1)),
    DropoffY_12=pd.NamedAgg(column='DropoffY',aggfunc = lambda t: t.head(12).tail(1)),
    Cost_12=pd.NamedAgg(column='Cost',aggfunc = lambda t: t.head(12).tail(1)),
    CollectedFareType_12=pd.NamedAgg(column='CollectedFareType',aggfunc = lambda t: t.head(12).tail(1)),
    TotalTripTimeSeconds_12=pd.NamedAgg(column='TotalTripTimeSeconds',aggfunc = lambda t: t.head(12).tail(1)),
    CustomerID_13=pd.NamedAgg(column='CustomerID',aggfunc = lambda t: t.head(13).tail(1)),
    StartDateTime_13=pd.NamedAgg(column='StartDateTime',aggfunc = lambda t: t.head(13).tail(1)),
    PickupX_13=pd.NamedAgg(column='PickupX',aggfunc = lambda t: t.head(13).tail(1)),
    PickupY_13=pd.NamedAgg(column='PickupY',aggfunc = lambda t: t.head(13).tail(1)),
    EndDateTime_13=pd.NamedAgg(column='EndDateTime',aggfunc = lambda t: t.head(13).tail(1)),
    DropoffX_13=pd.NamedAgg(column='DropoffX',aggfunc = lambda t: t.head(13).tail(1)),
    DropoffY_13=pd.NamedAgg(column='DropoffY',aggfunc = lambda t: t.head(13).tail(1)),
    Cost_13=pd.NamedAgg(column='Cost',aggfunc = lambda t: t.head(13).tail(1)),
    CollectedFareType_13=pd.NamedAgg(column='CollectedFareType',aggfunc = lambda t: t.head(13).tail(1)),
    TotalTripTimeSeconds_13=pd.NamedAgg(column='TotalTripTimeSeconds',aggfunc = lambda t: t.head(13).tail(1)),
    CustomerID_14=pd.NamedAgg(column='CustomerID',aggfunc = lambda t: t.head(14).tail(1)),
    StartDateTime_14=pd.NamedAgg(column='StartDateTime',aggfunc = lambda t: t.head(14).tail(1)),
    PickupX_14=pd.NamedAgg(column='PickupX',aggfunc = lambda t: t.head(14).tail(1)),
    PickupY_14=pd.NamedAgg(column='PickupY',aggfunc = lambda t: t.head(14).tail(1)),
    EndDateTime_14=pd.NamedAgg(column='EndDateTime',aggfunc = lambda t: t.head(14).tail(1)),
    DropoffX_14=pd.NamedAgg(column='DropoffX',aggfunc = lambda t: t.head(14).tail(1)),
    DropoffY_14=pd.NamedAgg(column='DropoffY',aggfunc = lambda t: t.head(14).tail(1)),
    Cost_14=pd.NamedAgg(column='Cost',aggfunc = lambda t: t.head(14).tail(1)),
    CollectedFareType_14=pd.NamedAgg(column='CollectedFareType',aggfunc = lambda t: t.head(14).tail(1)),
    TotalTripTimeSeconds_14=pd.NamedAgg(column='TotalTripTimeSeconds',aggfunc = lambda t: t.head(14).tail(1)),
    CustomerID_15=pd.NamedAgg(column='CustomerID',aggfunc = lambda t: t.head(15).tail(1)),
    StartDateTime_15=pd.NamedAgg(column='StartDateTime',aggfunc = lambda t: t.head(15).tail(1)),
    PickupX_15=pd.NamedAgg(column='PickupX',aggfunc = lambda t: t.head(15).tail(1)),
    PickupY_15=pd.NamedAgg(column='PickupY',aggfunc = lambda t: t.head(15).tail(1)),
    EndDateTime_15=pd.NamedAgg(column='EndDateTime',aggfunc = lambda t: t.head(15).tail(1)),
    DropoffX_15=pd.NamedAgg(column='DropoffX',aggfunc = lambda t: t.head(15).tail(1)),
    DropoffY_15=pd.NamedAgg(column='DropoffY',aggfunc = lambda t: t.head(15).tail(1)),
    Cost_15=pd.NamedAgg(column='Cost',aggfunc = lambda t: t.head(15).tail(1)),
    CollectedFareType_15=pd.NamedAgg(column='CollectedFareType',aggfunc = lambda t: t.head(15).tail(1)),
    TotalTripTimeSeconds_15=pd.NamedAgg(column='TotalTripTimeSeconds',aggfunc = lambda t: t.head(15).tail(1)),
    CustomerID_16=pd.NamedAgg(column='CustomerID',aggfunc = lambda t: t.head(16).tail(1)),
    StartDateTime_16=pd.NamedAgg(column='StartDateTime',aggfunc = lambda t: t.head(16).tail(1)),
    PickupX_16=pd.NamedAgg(column='PickupX',aggfunc = lambda t: t.head(16).tail(1)),
    PickupY_16=pd.NamedAgg(column='PickupY',aggfunc = lambda t: t.head(16).tail(1)),
    EndDateTime_16=pd.NamedAgg(column='EndDateTime',aggfunc = lambda t: t.head(16).tail(1)),
    DropoffX_16=pd.NamedAgg(column='DropoffX',aggfunc = lambda t: t.head(16).tail(1)),
    DropoffY_16=pd.NamedAgg(column='DropoffY',aggfunc = lambda t: t.head(16).tail(1)),
    Cost_16=pd.NamedAgg(column='Cost',aggfunc = lambda t: t.head(16).tail(1)),
    CollectedFareType_16=pd.NamedAgg(column='CollectedFareType',aggfunc = lambda t: t.head(16).tail(1)),
    TotalTripTimeSeconds_16=pd.NamedAgg(column='TotalTripTimeSeconds',aggfunc = lambda t: t.head(16).tail(1)),
    CustomerID_17=pd.NamedAgg(column='CustomerID',aggfunc = lambda t: t.head(17).tail(1)),
    StartDateTime_17=pd.NamedAgg(column='StartDateTime',aggfunc = lambda t: t.head(17).tail(1)),
    PickupX_17=pd.NamedAgg(column='PickupX',aggfunc = lambda t: t.head(17).tail(1)),
    PickupY_17=pd.NamedAgg(column='PickupY',aggfunc = lambda t: t.head(17).tail(1)),
    EndDateTime_17=pd.NamedAgg(column='EndDateTime',aggfunc = lambda t: t.head(17).tail(1)),
    DropoffX_17=pd.NamedAgg(column='DropoffX',aggfunc = lambda t: t.head(17).tail(1)),
    DropoffY_17=pd.NamedAgg(column='DropoffY',aggfunc = lambda t: t.head(17).tail(1)),
    Cost_17=pd.NamedAgg(column='Cost',aggfunc = lambda t: t.head(17).tail(1)),
    CollectedFareType_17=pd.NamedAgg(column='CollectedFareType',aggfunc = lambda t: t.head(17).tail(1)),
    TotalTripTimeSeconds_17=pd.NamedAgg(column='TotalTripTimeSeconds',aggfunc = lambda t: t.head(17).tail(1)),
    CustomerID_18=pd.NamedAgg(column='CustomerID',aggfunc = lambda t: t.head(18).tail(1)),
    StartDateTime_18=pd.NamedAgg(column='StartDateTime',aggfunc = lambda t: t.head(18).tail(1)),
    PickupX_18=pd.NamedAgg(column='PickupX',aggfunc = lambda t: t.head(18).tail(1)),
    PickupY_18=pd.NamedAgg(column='PickupY',aggfunc = lambda t: t.head(18).tail(1)),
    EndDateTime_18=pd.NamedAgg(column='EndDateTime',aggfunc = lambda t: t.head(18).tail(1)),
    DropoffX_18=pd.NamedAgg(column='DropoffX',aggfunc = lambda t: t.head(18).tail(1)),
    DropoffY_18=pd.NamedAgg(column='DropoffY',aggfunc = lambda t: t.head(18).tail(1)),
    Cost_18=pd.NamedAgg(column='Cost',aggfunc = lambda t: t.head(18).tail(1)),
    CollectedFareType_18=pd.NamedAgg(column='CollectedFareType',aggfunc = lambda t: t.head(18).tail(1)),
    TotalTripTimeSeconds_18=pd.NamedAgg(column='TotalTripTimeSeconds',aggfunc = lambda t: t.head(18).tail(1))
)
    
    
df_routes.to_csv("wide_raw_backup.csv")

In [9]:
# clearing out duplicate values

df_routes.loc[df_routes['TotalCustomers'] < 18 , ['CustomerID_18','StartDateTime_18','PickupX_18','PickupY_18','EndDateTime_18','DropoffX_18','DropoffY_18','Cost_18','CollectedFareType_18','TotalTripTimeSeconds_18']] = pd.NA
df_routes.loc[df_routes['TotalCustomers'] < 17 , ['CustomerID_17','StartDateTime_17','PickupX_17','PickupY_17','EndDateTime_17','DropoffX_17','DropoffY_17','Cost_17','CollectedFareType_17','TotalTripTimeSeconds_17']] = pd.NA
df_routes.loc[df_routes['TotalCustomers'] < 16 , ['CustomerID_16','StartDateTime_16','PickupX_16','PickupY_16','EndDateTime_16','DropoffX_16','DropoffY_16','Cost_16','CollectedFareType_16','TotalTripTimeSeconds_16']] = pd.NA
df_routes.loc[df_routes['TotalCustomers'] < 15 , ['CustomerID_15','StartDateTime_15','PickupX_15','PickupY_15','EndDateTime_15','DropoffX_15','DropoffY_15','Cost_15','CollectedFareType_15','TotalTripTimeSeconds_15']] = pd.NA
df_routes.loc[df_routes['TotalCustomers'] < 14 , ['CustomerID_14','StartDateTime_14','PickupX_14','PickupY_14','EndDateTime_14','DropoffX_14','DropoffY_14','Cost_14','CollectedFareType_14','TotalTripTimeSeconds_14']] = pd.NA
df_routes.loc[df_routes['TotalCustomers'] < 13 , ['CustomerID_13','StartDateTime_13','PickupX_13','PickupY_13','EndDateTime_13','DropoffX_13','DropoffY_13','Cost_13','CollectedFareType_13','TotalTripTimeSeconds_13']] = pd.NA
df_routes.loc[df_routes['TotalCustomers'] < 12 , ['CustomerID_12','StartDateTime_12','PickupX_12','PickupY_12','EndDateTime_12','DropoffX_12','DropoffY_12','Cost_12','CollectedFareType_12','TotalTripTimeSeconds_12']] = pd.NA
df_routes.loc[df_routes['TotalCustomers'] < 11 , ['CustomerID_11','StartDateTime_11','PickupX_11','PickupY_11','EndDateTime_11','DropoffX_11','DropoffY_11','Cost_11','CollectedFareType_11','TotalTripTimeSeconds_11']] = pd.NA
df_routes.loc[df_routes['TotalCustomers'] < 10 , ['CustomerID_10','StartDateTime_10','PickupX_10','PickupY_10','EndDateTime_10','DropoffX_10','DropoffY_10','Cost_10','CollectedFareType_10','TotalTripTimeSeconds_10']] = pd.NA
df_routes.loc[df_routes['TotalCustomers'] < 9 , ['CustomerID_9','StartDateTime_9','PickupX_9','PickupY_9','EndDateTime_9','DropoffX_9','DropoffY_9','Cost_9','CollectedFareType_9','TotalTripTimeSeconds_9']] = pd.NA
df_routes.loc[df_routes['TotalCustomers'] < 8 , ['CustomerID_8','StartDateTime_8','PickupX_8','PickupY_8','EndDateTime_8','DropoffX_8','DropoffY_8','Cost_8','CollectedFareType_8','TotalTripTimeSeconds_8']] = pd.NA
df_routes.loc[df_routes['TotalCustomers'] < 7 , ['CustomerID_7','StartDateTime_7','PickupX_7','PickupY_7','EndDateTime_7','DropoffX_7','DropoffY_7','Cost_7','CollectedFareType_7','TotalTripTimeSeconds_7']] = pd.NA
df_routes.loc[df_routes['TotalCustomers'] < 6 , ['CustomerID_6','StartDateTime_6','PickupX_6','PickupY_6','EndDateTime_6','DropoffX_6','DropoffY_6','Cost_6','CollectedFareType_6','TotalTripTimeSeconds_6']] = pd.NA
df_routes.loc[df_routes['TotalCustomers'] < 5 , ['CustomerID_5','StartDateTime_5','PickupX_5','PickupY_5','EndDateTime_5','DropoffX_5','DropoffY_5','Cost_5','CollectedFareType_5','TotalTripTimeSeconds_5']] = pd.NA
df_routes.loc[df_routes['TotalCustomers'] < 4 , ['CustomerID_4','StartDateTime_4','PickupX_4','PickupY_4','EndDateTime_4','DropoffX_4','DropoffY_4','Cost_4','CollectedFareType_4','TotalTripTimeSeconds_4']] = pd.NA
df_routes.loc[df_routes['TotalCustomers'] < 3 , ['CustomerID_3','StartDateTime_3','PickupX_3','PickupY_3','EndDateTime_3','DropoffX_3','DropoffY_3','Cost_3','CollectedFareType_3','TotalTripTimeSeconds_3']] = pd.NA
df_routes.loc[df_routes['TotalCustomers'] < 2 , ['CustomerID_2','StartDateTime_2','PickupX_2','PickupY_2','EndDateTime_2','DropoffX_2','DropoffY_2','Cost_2','CollectedFareType_2','TotalTripTimeSeconds_2']] = pd.NA

In [14]:
# output wide results to csv
df_routes.to_csv("processed_aar_trips_wide.csv")

In [16]:
# output long results to csv
header = ['RunActualID','VehicleID','CustomerID','CustomerSex','CustomerBirthDate','TripRank','TripDate','StartDateTime','EndDateTime','TotalTripTimeSeconds','Cost','CollectedFareType','PickupX','PickupY','DropoffX','DropoffY']
df.to_csv("processed_aar_trips_long.csv", index=False, columns=header)

In [17]:
# output customer key to csv
print(df['CustomerID'].nunique())
customer_key = df.groupby(['CustomerID']).agg(
    CustomerSex=pd.NamedAgg(column='CustomerSex', aggfunc='max'),
    CustomerBirthDate=pd.NamedAgg(column='CustomerBirthDate', aggfunc='max'),
    TotalTrips=pd.NamedAgg(column='CustomerID', aggfunc='count')
    )
customer_key.to_csv("customer_key.csv")

10215


In [13]:
# checking for dropoffs at same time and place
print(f"""Routes where first and second customer have the same end date time: {
    df_routes.loc[(df_routes['EndDateTime_1'] == df_routes['EndDateTime_2'])
        & (df_routes['DropoffX_1'].notnull())
        & (df_routes['DropoffX_2'].notnull())].shape[0]}.""")
print(f"""Routes with same end time and exact same dropoff coordinates: {
    df_routes.loc[(df_routes['EndDateTime_1'] == df_routes['EndDateTime_2'])
        & (df_routes['DropoffX_1'].notnull())
        & (df_routes['DropoffX_2'].notnull())
        & (df_routes['DropoffX_1'] == df_routes['DropoffX_2'])
        & (df_routes['DropoffY_1'] == df_routes['DropoffY_2'])
        ].shape[0]}.""")

print(f"""Routes with same end time and same dropoff coordinates, rounded to 5 decimal places: {
    df_routes.loc[(df_routes['EndDateTime_1'] == df_routes['EndDateTime_2'])
        & (df_routes['DropoffX_1'].notnull())
        & (df_routes['DropoffX_2'].notnull())
        & (df_routes['DropoffX_1'].round(5) == df_routes['DropoffX_2'].round(5))
        & (df_routes['DropoffY_1'].round(5) == df_routes['DropoffY_2'].round(5))
        ].shape[0]}""")

print(f"""Routes with same end time and same dropoff coordinates, rounded to 4 decimal places: {
    df_routes.loc[(df_routes['EndDateTime_1'] == df_routes['EndDateTime_2'])
        & (df_routes['DropoffX_1'].notnull())
        & (df_routes['DropoffX_2'].notnull())
        & (df_routes['DropoffX_1'].round(4) == df_routes['DropoffX_2'].round(4))
        & (df_routes['DropoffY_1'].round(4) == df_routes['DropoffY_2'].round(4))
        ].shape[0]}""")

print(f"""Routes with same end time and same dropoff coordinates, rounded to 3 decimal places: {
    df_routes.loc[(df_routes['EndDateTime_1'] == df_routes['EndDateTime_2'])
        & (df_routes['DropoffX_1'].notnull())
        & (df_routes['DropoffX_2'].notnull())
        & (df_routes['DropoffX_1'].round(3) == df_routes['DropoffX_2'].round(3))
        & (df_routes['DropoffY_1'].round(3) == df_routes['DropoffY_2'].round(3))
        ].shape[0]}""")



Routes where first and second customer have the same end date time: 5376.
Routes with same end time and exact same dropoff coordinates: 4684.
Routes with same end time and same dropoff coordinates, rounded to 5 decimal places: 5178
Routes with same end time and same dropoff coordinates, rounded to 4 decimal places: 5206
Routes with same end time and same dropoff coordinates, rounded to 3 decimal places: 5335
